In [1]:
import os
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer\\research'

In [2]:
os.chdir("..")

In [3]:
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer'

In [5]:
from pathlib import Path

In [8]:
from dataclasses import dataclass
@dataclass
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int 
    save_steps: float
    gradient_accumulation_steps: int



In [9]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        create_directories([self.config.artifact_root],verbose=True)
    def get_model_trainer_config(self)->ModelTrainingConfig:
        config=self.config.model_trainer
        create_directories([config.root_dir],verbose=True)
        model_training_config=ModelTrainingConfig(root_dir=config.root_dir,data_path=config.data_path,model_ckpt=config.model_ckpt)
        return model_training_config
        

In [ ]:
# Training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    eval_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt['test'],eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
trainer.train()